In [113]:
import nltk
import pandas as pd
import requests, re
import numpy as np

In [114]:
print("numpy == ", np.__version__)
print("pandas == ", pd.__version__)
print("nltk == ", nltk.__version__)

numpy ==  1.22.4
pandas ==  1.3.5
nltk ==  3.7


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df =  pd.read_csv('/content/nyt.csv')

In [ ]:
df.columns

Index(['_id', 'link', 'published', 'author', 'category', 'credit', 'sumary',
       'title'],
      dtype='object')

In [ ]:
df.head(3)

,_id,link,published,author,category,credit,sumary,title
0,63f972ec60c21a53adcf41cb,https://www.nytimes.com/2023/02/24/business/ec...,2023-02-24 21:28:39,Jeanna Smialek and Ben Casselman,Economy,The New York Times,"Inflation is down from its peak last summer, b...",The Fed’s Preferred Inflation Gauge Sped Back Up
1,63f972ec60c21a53adcf41cd,https://www.nytimes.com/2023/02/23/business/ec...,2023-02-23 22:58:50,Noam Scheiber,Economy,Brittany Greeson for The New York Times,After barring Starbucks from firing any U.S.-b...,Judge Scales Back Ruling Against Starbucks in ...
2,63f972ec60c21a53adcf41cf,https://www.nytimes.com/2023/02/23/business/ec...,2023-02-23 10:00:20,Ana Swanson and Don Clark,Economy,Philip Cheung for The New York Times,"Semiconductor companies, which united to get t...",Chip Makers Turn Cutthroat in Fight for Share ...


In [ ]:
corpus =  []
corpus =  [document for document in df.sumary]

In [ ]:
#afficher un document
corpus[0]

'Inflation is down from its peak last summer, but recent readings have shown substantial and surprising staying power.'

## Presentation des differentes approches pour la phase d'analyse de sentiment 

- Approche 1 : Approche classique de text mining par usage d'un jeu de données de finance préalablement labelisé.



On decide concerant cette approche de s'orienter vers a dataset de commentaires a caractere financiers avec les polarité deja renseigné pour la construction de notre modele de classification. 
le dataset source est disponible a l'adresse : <a href="https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news?resource=download">link</a>

In [115]:
dataset_de_base =  pd.read_csv("/content/drive/MyDrive/dataset_kaggle_sentiment_analysis.csv")

In [116]:
dataset_de_base.columns

Index(['polarity', 'comments'], dtype='object')

In [117]:
data_ref =  pd.DataFrame()

In [118]:
data_ref['polarity'] = [dataset_de_base.iloc[:, 0][i] for i in range(4845)]
data_ref['comments'] =  [dataset_de_base.iloc[:, 1][i] for i in range(4845)]

In [119]:
data_ref.head()

,polarity,comments
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [ ]:
data_ref.shape

(4845, 2)

In [120]:
data_ref['polarity'].value_counts()

neutral     2878
positive    1363
negative     604
Name: polarity, dtype: int64

**Desequilibre de classe tres observé**

On decide de restreindre notre jeu de données aux observations aux labels **negative** et **positive**.


In [121]:
data_ref_negatif =  data_ref[data_ref['polarity'] == "negative"]
data_ref_positif =  data_ref[data_ref['polarity'] == "positive"].iloc[:data_ref_negatif.shape[0] + 20]

In [122]:
#reconstruction du jeu de donnée de reference pour coller un temps soit peu a la problematique
data_ref =  data_ref_negatif
data_ref = data_ref.append(data_ref_positif, ignore_index=True)

In [123]:
data_ref.head(3)

,polarity,comments
0,negative,The international electronic industry company ...
1,negative,A tinyurl link takes users to a scamming site ...
2,negative,"Compared with the FTSE 100 index , which rose ..."


In [124]:
#melanger les lignes de notre nouveau jeu de données
data_ref = data_ref.sample(frac=1).reset_index(drop=True)

In [125]:
data_ref.head(3)

,polarity,comments
0,positive,"In the second quarter of 2010 , the group 's n..."
1,positive,Ruukki signed the contract with Norwegian comp...
2,negative,Incap Furniture is presently negotiating staff...


In [126]:
labels =  data_ref['polarity']

In [127]:
corpus = data_ref['comments'].tolist()

In [128]:
#taille du corpus
print(len(corpus))

#uniformisation de la case
corpus =  [doc.lower() for doc in corpus]

1228


In [129]:
corpus[0]

"in the second quarter of 2010 , the group 's net profit rose to eur 3.1 million from eur 2.5 million in april-june 2009 ."

In [130]:
#suppression de la ponctuation 
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [131]:
# apres suppresion reconstitution du document par la methode join 
corpus =  [" ".join(tokenizer.tokenize(doc)) for doc in corpus ]

In [132]:
corpus[0]

'in the second quarter of 2010 the group s net profit rose to eur 3 1 million from eur 2 5 million in april june 2009'

In [133]:
# suppression des chiffres
def detector_number(char):
    """ tester si la chaine est un nombre ou chiffre """
    try:
        test = eval(char)
        return True
    except:
        return False
    
def delete_number_on_doc(document):
    """ supprimer les données numeriques d'un document """
    document = document.split()
    document =  [word for word in document if detector_number(word) == False ]
    
    return " ".join(document)

In [134]:
corpus =  [delete_number_on_doc(doc) for doc in corpus]

In [135]:
corpus[0]

'in the second quarter of the group s net profit rose to eur million from eur million in april june'

In [136]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [137]:
#supression des stopwords
from nltk.corpus import stopwords
sw =  stopwords.words('english')

In [138]:
corpus_sw =  [" ".join([ word for word in doc.split() if word not in sw]) for doc in corpus]

In [139]:
corpus_sw[0]

'second quarter group net profit rose eur million eur million april june'

In [140]:
#etape de tokenization
corpus_token =  [doc.split() for doc in corpus_sw]

In [141]:
corpus_token[0]

['second',
 'quarter',
 'group',
 'net',
 'profit',
 'rose',
 'eur',
 'million',
 'eur',
 'million',
 'april',
 'june']

In [142]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api

In [143]:
pretrained_model = api.load("glove-wiki-gigaword-50")

[================================================--] 96.9% 63.9/66.0MB downloaded


In [144]:
#conversion des document en vecteur 
def doc2vec(doc: list) -> np.ndarray:
    """Convert a document to a vector."""
    doc_vector = np.zeros(50)
    for word in doc.split():
        if word in pretrained_model:
            doc_vector += pretrained_model[word]
    return doc_vector/len(doc_vector)

In [145]:
#test
print(corpus_token[0][0], " --> ", doc2vec(corpus_token[0][0]))

second  -->  [-5.96180022e-03  5.61380029e-03  1.74204007e-03  1.08910000e-02
  1.40006006e-02  8.95560026e-03 -1.45130002e-02  1.24618006e-02
  1.37725996e-05 -4.21120003e-04  5.05580008e-03 -1.11521995e-02
 -2.19939995e-02  7.88040012e-04  1.05333996e-02 -1.26038003e-02
 -3.73299986e-03 -1.58555996e-02 -3.19239998e-02  5.12380004e-03
 -3.87879997e-03 -8.61980021e-03 -3.44080001e-03 -3.22600007e-03
  3.01899999e-03 -3.21059990e-02  3.17380011e-03 -8.86759996e-03
 -2.71519989e-03 -8.75200033e-04  6.94040012e-02  2.82400012e-03
 -2.27840006e-03  1.09024000e-02  9.17859972e-03 -3.71219993e-03
  1.12188005e-02  1.75911999e-02  1.03498006e-05 -1.14268005e-02
 -2.35540003e-03 -2.71880001e-03 -7.78859973e-03 -1.58534002e-02
 -1.14657998e-02  7.64180005e-03  9.11679983e-03 -9.67920005e-03
  6.99320018e-03 -5.98160028e-03]


In [146]:
features = np.array([doc2vec(x) for x in corpus])

In [147]:
features.shape, labels.shape

((1228, 50), (1228,))

In [148]:
#modele de regression logistic
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold

In [149]:
seed =  0
reglog = LogisticRegression(random_state=seed)
rf = RandomForestClassifier(random_state =seed)

In [150]:
#estima des capacites predictrices de notre modele de regression logistique
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=120)


def estimation_par_crossvalidation(model, X_train, y_train):
  """ Fonction d'evaluation par cross validation d'un modele """
  
  cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
  n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
  print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [151]:
#modele de regression logistique
estimation_par_crossvalidation(reglog, X_train, y_train)

#modele de randomForest
estimation_par_crossvalidation(rf, X_train, y_train)

Mean Accuracy: 0.693 (0.031)
Mean Accuracy: 0.703 (0.025)


Nous decidons d'opter pour le modele de *regression logistique* qui a des resultats trop similaire a un modele tres elaborés comme le randomForest, l'objectif etant de maximiser sur **l'explicabilité du modele**.



Bien sure un travail de modelisation plus elaboré pourra etre realisé avec des methodes *undersampling*, *oversampling* pour gérer le desequilibre de classe assez existant dans notre dataset de reference. 

In [152]:
reglog.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [153]:
#test sur la premiere observation de notre jeu de données de test
reglog.predict(X_test)[0]

'negative'

In [154]:
reglog.score(X_test,y_test)

0.7560975609756098

In [156]:
# suppression des chiffres
def detector_number(char):
    """ tester si la chaine est un nombre ou chiffre """
    try:
        test = eval(char)
        return True
    except:
        return False
    
def delete_number_on_doc(document):
    """ supprimer les données numeriques d'un document """
    document = document.split()
    document =  [word for word in document if detector_number(word) == False ]
    
    return " ".join(document)



#preprocessing_modele_1
def preproc_modele_1(observation):
  """ Depile tout la pipeline de pretraitement des données """

  #uniformisation de la case
  observation =  observation.lower()


  #supression des caracterere speciaux
  tokenizer = RegexpTokenizer(r'\w+')
  observation =  " ".join(tokenizer.tokenize(observation))

  #suppresion des données a caractere numeriques
  observation = delete_number_on_doc(observation)

  #suppresion des stopwords
  observation =  " ".join([ word for word in observation.split() if word not in sw])

  #tokenization
  observation = observation.split()

  return observation


#conversion des document en vecteur 
def doc2vec(doc: list) -> np.ndarray:
    """Convert a document to a vector."""
    doc_vector = np.zeros(50)
    for word in doc.split():
        if word in pretrained_model:
            doc_vector += pretrained_model[word]
    return doc_vector/len(doc_vector)



def modele_1(model, observation):

  """ pipeline depuis l'etape de la vectorisation a la prediction avec le modele choisi """
    
  #vectorization
  vector =  np.array([doc2vec(observation)])
  
  #prediction
  prediction = model.predict(vector) 

  return prediction
    
  

In [157]:
#petit test
modele_1(reglog, 'churn crisis for UTB')

array(['negative'], dtype=object)

- Approche 2 : Methode de transfert learning avec le modele **Gradio** stocker sur *Huggin'Face* 


endpoint du modele:  <a href = https://huggingface.co/spaces/Amrrs/gradio-sentiment-analyzer>gradio</a>

In [158]:
def modele_2(articles):
  
  """ Pour le cadre de la classiffication decide d'opter pour du transfert learning sur un model d'analyse de sentiment stocker sur huggin'Face """
  
  #end point du modele gardio sentiment analyzer
  response = requests.post("https://amrrs-gradio-sentiment-analyzer.hf.space/run/predict", json={
	"data": [
		articles,
	  ]
  }).json()
  
  #recuperation du score dans la reponse json
  score =  eval(response['data'][0].split()[-1][:-2])
  
  #recuperation du label dans la reponse json
  label =  response['data'][0].split()[1].replace(',', '').replace("'","")
  
  return (label, score)

In [168]:
#petit test
modele_2("churn crisis for UTB")[0]

'NEGATIVE'

In [163]:
data_ref.columns

Index(['polarity', 'comments'], dtype='object')

In [166]:
#evaluons les performance du modele GRADIO
from sklearn.model_selection import train_test_split

X = data_ref['comments']
y = data_ref['polarity']

pred = []

In [167]:
y.unique()

array(['positive', 'negative'], dtype=object)

In [169]:
for document in X: 
  prediction = modele_2(document)[0].lower()
  pred.append(prediction)

In [171]:
pred[:3]

['positive', 'positive', 'negative']

In [178]:
len(pred)

1228

In [188]:
def acc_test(y_vrai, y_pred):
  """ Fonction pour calculer l'accuracy du modele en usant de ses predictions sur notre jeu de données """
  compt =  0
  for ii in range(len(y_vrai)):
    if y_vrai[ii] == y_pred[ii]:
      compt += 1
  return compt / len(y_vrai)

In [182]:
#evaluation de l'accuracy de notre gradio 
acc_test(y_vrai = y, y_pred = pred)

0.752442996742671

Accuracy evalué sur l'integralité du jeu de données de reference est relativement 

# Conclusion
 
Les modèles mis en competition ont des niveaux de precision relativement approximatif aux vues de notre jeu de données de reference on decide donc d'opter pour le modele **Gradio** dont les endpoint sont disponibles sur *Huggin'Face.*
Cependant on peut meme affirmer que des modeles pre-entrainés orientés **finance** serait une meilleur option avec notamment le celebre modele : <a href="https://huggingface.co/yiyanghkust/finbert-tone">finbert</a>
